# Immediate mangrove damage from cyclones
#### This notebook calculates the immediate mangrove damage following cyclones. It uses the mangrove cover for the year before and the year after the cyclone to create damage statistics.

**What does this notebook do?**

There are essentially two parts to this notebook:

The **first part** creates NetCDFs of mangrove cover damage for each Australian Albers tile affected by specified cyclones. 

1. Imports modules and sets up working folder
2. Sets up a list of cyclones, the years they occurred, the Australian Albers tiles affected and the wind speeds categories experienced
3. Sets up functions for generating mangrove cover damage statistics
4. Creates NetCDFs of mangrove damage for Australian Albers tiles that have been affected by the listed cyclones

The **second part** combines the mangrove damage statistics for each wind speed category within all Australian Albers tiles affected by certain cyclones, and plots and exports csv files of these statistics. The following section of this notebook uses the damage statistics created in previous cells to calculate the combined damage statistics for wind speed categories and the total mangrove damage for each cyclone.

5. Loops through each cyclone, windspeed category and Australian Albers tile to produce: 1) plots of mangrove cover damage for each wind speed category experienced during each cyclone and 2) csv files of the mangrove cover damage statistics
6. Uses the data created in previous cells to calculate the damage statistics for mangrove areas that have been subjected to certain wind speed categories, then combines, plots and produces csv files of the combined category data from all cyclones
7. Uses the data created in previous cells to calculate the total mangrove damage for each cyclone, then produces plots and csv files of these data

**Requirements and data sources:**

You need to run the following commands from the command line or a shell script prior to launching Jupyter notebooks from the same terminal so that the requiered libraries and paths are set:

`module use /g/data/v10/public/modules/modulefiles`

`module load dea`


**Author:** Sebastian Wong, Claire Krause, Emma Ai 

**Date:** February 2019

## Importing modules and setting up working directory
The cell below imports modules and identifies the directory of the working folder

In [1]:
# Import modules

%pylab notebook

import os
import os.path
import shutil
import re
import rsgislib
import rsgislib.vectorutils
import datacube
import sys
import rasterio
import time
import subprocess
import yaml
import matplotlib.pyplot
import xarray as xr
import pandas as pd
import geopandas as gpd
import osgeo.gdal as gdal
import numpy as np
from rasterstats import zonal_stats
from datacube.storage import netcdf_writer
from datacube.storage.storage import create_netcdf_storage_unit, write_dataset_to_netcdf
from os import path
from datacube.model import Measurement  
from datacube.model.utils import make_dataset, xr_apply, datasets_to_doc
from pathlib import Path
from datacube.api.query import query_group_by, query_geopolygon
from datacube.utils.geometry import CRS
from matplotlib.pyplot import subplots_adjust
from datacube.model import DatasetType as Product
from yaml import CSafeLoader as Loader, CSafeDumper as Dumper

# Identify the directory of the working forlder

os.chdir('/g/data/r78/sw4446/Mangrove_Change_on_Cyclone/')

Populating the interactive namespace from numpy and matplotlib


## Data input lists
Below are lists of 1) cyclones and the year they occurred (for input into the `AllCyclones` variable), 2) the Australian Albers tiles affected by each cyclone (for input into the `TilesForCyclones` variable), and 3) the wind speed categories experienced by each cyclone (for input into the `NumberofCategories` variable). The purpose of these lists are for easy repeatability and modifications of the scripts. Any cyclone, wind speed category and Australian Albers tile combination can be run simultaneously, by simply copying and pasting the lists, or parts of the lists, into the cell where the relevant variable is defined. (Alternatively, the complete lists are provided as comments in the code cell below. These lists, or parts of the lists, can be uncommented to run the desired cyclones/tile/category combination).


### All Cyclones 
Below is a list of all cyclones analysed during this study. These are for input into the `AllCyclones` variable. 

'IngridLF1': 2005, 'IngridLF2': 2005, 'IngridLF3': 2005, 'Larry': 2006, 'MonicaLF1': 2006, 'MonicaLF2': 2006, 'George': 2007, 'Laurence': 2009, 'Yasi': 2011, 'Ita': 2014, 'Lam': 2015, 'Marcia': 2015, 'NathanLF1': 2015, 'NathanLF2': 2015,'Debbie': 2017


### Australian Albers tile list for specific cyclones
Below is a list of Australian Albers tiles which have been subjected to specific cyclones. The Australian Albers tiles have been grouped into lists based on the specific cyclones that affected these tiles. The month and year of the cyclones have also been identified in the list. These are for input into the `TilesForCyclones` variabe

**IngridLF1 (03/2005):**
'12,-14', '12,-15', '12,-16', '13,-16', '14,-16', '12,-17', '13,-17', '14,-17', '14,-18', '14,-19'

**IngridLF2 (03/2005):**
'-2,-12', '-1,-12', '0,-12', '1,-12', '5,-12', '-3,-13', '-2,-13', '-1,-13', '0,-13', '1,-13', '2,-13', '3,-13', 
'4,-13', '5,-13', '-3,-14', '-2,-14', '-1,-14', '0,-14', '3,-14', '4,-14', '5,-14', '4,-15'

**IngridLF3 (03/2005):**
'-7,-15', '-6,-15', '-5,-15', '-8,-16', '-7,-16', '-6,-16', '-5,-16', '-8,-17', '-5,-17'

**Larry (03/2006):**
'14,-20', '14,-21', '15,-21', '14,-22', '15,-22'

**MonicaLF1 (04/2006):**
'12,-14', '12,-15', '12,-16', '13,-16', '14,-16', '12,-17', '13,-17', '14,-17', '14,-18', '14,-19'

**MonicaLF2 (04/2006):**
'-2,-12', '-1,-12', '0,-12', '1,-12', '5,-12', '-3,-13', '-2,-13', '-1,-13', '0,-13', '1,-13', '2,-13', '3,-13', '4,-13', '5,-13', '-3,-14', '-2,-14', '-1,-14', '0,-14', '3,-14', '4,-14', '5,-14', '4,-15'

**George (02-03/2007):**
'-13,-22', '-18,-23', '-17,-23', '-16,-23', '-15,-23', '-14,-23', '-13,-23', '-18,-24', '-17,-24', '-16,-24', '-15,-24', '-18,-25', '-17,-25'

**Laurence (12/2009):**
'-7,-15', '-6,-15', '-5,-15', '-8,-16', '-7,-16', '-6,-16', '-5,-16', '-9,-17', '-8,-17', '-7,-17', '-10,-18', '-9,-18', '-8,-18', '-11,-19', '-10,-19', '-9,-19', '-11,-20', '-11,-21'

**Yasi (01-02/2011):**
'14,-20', '14,-21', '15,-21', '14,-22', '15,-22'

**Ita (04/2014):**
'12,-16', '13,-16', '14,-16', '12,-17', '13,-17', '14,-17', '14,-18', '14,-19', '14,-20', '14,-21'

**Lam (02/2015):**
'-2,-12', '-1,-12', '0,-12', '1,-12', '5,-12', '-3,-13', '-2,-13', '-1,-13', '0,-13', '1,-13', '2,-13', '3,-13', '4,-13', '5,-13', '-3,-14', '-2,-14', '-1,-14', '0,-14', '3,-14', '4,-14', '5,-14', '4,-15'

**Marcia (02-03/2015):**
'17,-25', '17,-26', '18,-26', '19,-26', '18,-27', '19,-27', '18,-28', '19,-28', '19,-29', '20,-29' 

**NathanLF1 (03/2015):**
'12,-16', '13,-16', '14,-16', '12,-17', '13,-17', '14,-17', '14,-18', '14,-19'

**NathanLF2 (03/2015):**
'0,-12', '1,-12', '5,-12', '0,-13', '1,-13', '2,-13', '3,-13', '4,-13', '5,-13', '0,-14', '3,-14', '4,-14', '5,-14', '4,-15', '5,-15', '4,-16', '5,-16'

**Debbie (03-04/2017):**
'16,-23','17,-23', '17,-24'


### Wind speed categories for all cyclones
Below is a list of wind speed categories for all cyclones analysed during this study. These are for input into the `NumberofCategories` variable.

'IngridLF1': ['C1', 'C2', 'C3'], 'Larry': ['C1', 'C2', 'C3', 'C4', 'C5']}, 'IngridLF2': ['C1', 'C2', 'C3', 'C4', 'C5'], 'IngridLF3': ['C1', 'C2', 'C3', 'C4'], 'MonicaLF1': ['C1', 'C2', 'C3'], 'MonicaLF2': ['C1', 'C2', 'C3', 'C4', 'C5'], 'George': ['C1', 'C2', 'C3', 'C4', 'C5'], 'Laurence': ['C1', 'C2', 'C3', 'C4', 'C5'], 'Yasi': ['C2', 'C3', 'C4', 'C5'], 'Ita' :['C1', 'C2', 'C3', 'C4'], 'Lam': ['C1', 'C2', 'C3', 'C4'], 'Marcia': ['C1', 'C2', 'C3', 'C4', 'C5'], 'NathanLF1': ['C1', 'C2', 'C3'], 'NathanLF2': ['C1', 'C2'], 'Debbie': ['C2', 'C3', 'C4', 'C5']}

In [2]:
AllCyclones = { 
                'IngridLF1': 2005,
#                 'IngridLF2': 2005,
#                 'IngridLF3': 2005, 
#                 'Larry': 2006, 
#                 'MonicaLF1': 2006,
#                 'MonicaLF2': 2006, 
#                 'George': 2007, 
#                 'Laurence': 2009, 
#                 'Yasi': 2011, 
#                 'Ita': 2014, 
#                 'Lam': 2015, 
#                 'Marcia': 2015, 
#                 'NathanLF1': 2015, 
#                 'NathanLF2': 2015, 
#                 'Debbie': 2017   
              }

# Below is a copy of the full cyclone (year) list. This list (or part of it) can be copied an pasted above and 
#uncommented depending on which cyclone(s) you chose to run.

#                'IngridLF1': 2005,
#                'IngridLF2': 2005,
#                'IngridLF3': 2005, 
#                'Larry': 2006, 
#                'MonicaLF1': 2006,
#                'MonicaLF2': 2006, 
#                'George': 2007, 
#                'Laurence': 2009, 
#                'Yasi': 2011, 
#                'Ita': 2014, 
#                'Lam': 2015, 
#                'Marcia': 2015, 
#                'NathanLF1': 2015, 
#                'NathanLF2': 2015, 
#                'Debbie': 2017
                

TilesForCyclones =  {  
                        'IngridLF1': ['12,-14', '12,-15', '12,-16', '13,-16', '14,-16', '12,-17', '13,-17', 
                                  '14,-17', '14,-18', '14,-19'],
#                         'IngridLF2': ['-2,-12', '-1,-12', '0,-12', '1,-12', '5,-12', '-3,-13', '-2,-13', '-1,-13', '0,-13',
#                                    '1,-13', '2,-13', '3,-13', '4,-13', '5,-13', '-3,-14', '-2,-14', '-1,-14', '0,-14', 
#                                    '3,-14', '4,-14', '5,-14', '4,-15'],
#                         'IngridLF3': ['-7,-15', '-6,-15', '-5,-15', '-8,-16', '-7,-16', '-6,-16', '-5,-16', '-8,-17',
#                                    '-5,-17'],
#                         'Larry': ['14,-20', '14,-21', '15,-21', '14,-22', '15,-22'],
#                         'MonicaLF1': ['12,-14', '12,-15', '12,-16', '13,-16', '14,-16', '12,-17', '13,-17', '14,-17', 
#                                      '14,-18', '14,-19'],
#                         'MonicaLF2': ['-2,-12', '-1,-12', '0,-12', '1,-12', '5,-12', '-3,-13', '-2,-13', '-1,-13', '0,-13',
#                                    '1,-13', '2,-13', '3,-13', '4,-13', '5,-13', '-3,-14', '-2,-14', '-1,-14', '0,-14',
#                                    '3,-14', '4,-14', '5,-14', '4,-15'],
#                         'George': ['-16,-23', '-15,-23', '-14,-23', '-13,-23', '-17,-24', '-16,-24', '-15,-24', '-18,-25',
#                                    '-17,-25'],
#                         'Laurence': ['-7,-15', '-6,-15', '-5,-15', '-8,-16', '-7,-16', '-6,-16', '-5,-16', '-9,-17', 
#                                   '-8,-17', '-7,-17', '-10,-18', '-9,-18', '-8,-18', '-11,-19', '-10,-19', '-9,-19',
#                                   '-11,-20', '-11,-21'],
#                         'Yasi': ['14,-20', '14,-21', '15,-21', '14,-22', '15,-22'],
#                         'Ita': ['12,-16', '13,-16', '14,-16', '12,-17', '13,-17', '14,-17', '14,-18', '14,-19', '14,-20',
#                              '14,-21'],
#                         'Lam': ['-2,-12', '-1,-12', '0,-12', '1,-12', '5,-12', '-3,-13', '-2,-13', '-1,-13', '0,-13', 
#                              '1,-13', '2,-13', '3,-13', '4,-13', '5,-13', '-3,-14', '-2,-14', '-1,-14', '0,-14', '3,-14',
#                             '4,-14', '5,-14', '4,-15'],
#                         'Marcia': ['17,-25', '17,-26', '18,-26', '19,-26', '18,-27', '19,-27', '19,-28', '19,-29',
#                                 '20,-29' ], 
#                         'NathanLF1': ['12,-16', '13,-16', '14,-16', '12,-17', '13,-17', '14,-17', '14,-18', '14,-19'], 
#                         'NathanLF2': ['0,-12', '1,-12', '5,-12', '0,-13', '1,-13', '2,-13', '3,-13', '4,-13', '5,-13', 
#                                    '0,-14', '3,-14', '4,-14', '5,-14', '4,-15', '5,-15', '4,-16', '5,-16'],
#                         'Debbie': ['16,-23','17,-23', '17,-24']
                    }
                                        
# Below is a copy of the full cyclone (Australian Albers tile) list. This list (or part of it) can be copied an pasted
# above and uncommented depending on which cyclone(s) you chose to run.

#                       'IngridLF1': ['12,-14', '12,-15', '12,-16', '13,-16', '14,-16', '12,-17', '13,-17', 
#                                  '14,-17', '14,-18', '14,-19'],
#                       'IngridLF2': ['-2,-12', '-1,-12', '0,-12', '1,-12', '5,-12', '-3,-13', '-2,-13', '-1,-13', '0,-13',
#                                   '1,-13', '2,-13', '3,-13', '4,-13', '5,-13', '-3,-14', '-2,-14', '-1,-14', '0,-14', 
#                                   '3,-14', '4,-14', '5,-14', '4,-15'],
#                       'IngridLF3': ['-7,-15', '-6,-15', '-5,-15', '-8,-16', '-7,-16', '-6,-16', '-5,-16', '-8,-17',
#                                   '-5,-17'],
#                       'Larry': ['14,-20', '14,-21', '15,-21', '14,-22', '15,-22'],
#                       'MonicaLF1': ['12,-14', '12,-15', '12,-16', '13,-16', '14,-16', '12,-17', '13,-17', '14,-17', 
#                                   '14,-18', '14,-19'],
#                       'MonicaLF2': ['-2,-12', '-1,-12', '0,-12', '1,-12', '5,-12', '-3,-13', '-2,-13', '-1,-13', '0,-13',
#                                   '1,-13', '2,-13', '3,-13', '4,-13', '5,-13', '-3,-14', '-2,-14', '-1,-14', '0,-14',
#                                   '3,-14', '4,-14', '5,-14', '4,-15'],
#                       'George': ['-16,-23', '-15,-23', '-14,-23', '-13,-23', '-17,-24', '-16,-24', '-15,-24', '-18,-25',
#                                   '-17,-25'],
#                       'Laurence': ['-7,-15', '-6,-15', '-5,-15', '-8,-16', '-7,-16', '-6,-16', '-5,-16', '-9,-17', 
#                                  '-8,-17', '-7,-17', '-10,-18', '-9,-18', '-8,-18', '-11,-19', '-10,-19', '-9,-19',
#                                  '-11,-20', '-11,-21'],
#                       'Yasi': ['14,-20', '14,-21', '15,-21', '14,-22', '15,-22'],
#                       'Ita': ['12,-16', '13,-16', '14,-16', '12,-17', '13,-17', '14,-17', '14,-18', '14,-19', '14,-20',
#                             '14,-21'],
#                       'Lam': ['-2,-12', '-1,-12', '0,-12', '1,-12', '5,-12', '-3,-13', '-2,-13', '-1,-13', '0,-13', 
#                             '1,-13', '2,-13', '3,-13', '4,-13', '5,-13', '-3,-14', '-2,-14', '-1,-14', '0,-14', '3,-14',
#                             '4,-14', '5,-14', '4,-15'],
#                       'Marcia': ['17,-25', '17,-26', '18,-26', '19,-26', '18,-27', '19,-27', '19,-28', '19,-29',
#                                '20,-29' ], 
#                       'NathanLF1': ['12,-16', '13,-16', '14,-16', '12,-17', '13,-17', '14,-17', '14,-18', '14,-19'], 
#                       'NathanLF2': ['0,-12', '1,-12', '5,-12', '0,-13', '1,-13', '2,-13', '3,-13', '4,-13', '5,-13', 
#                                   '0,-14', '3,-14', '4,-14', '5,-14', '4,-15', '5,-15', '4,-16', '5,-16'],
#                       'Debbie': ['16,-23','17,-23', '17,-24']


NumberofCategories =    { 
                        'IngridLF1': ['C1', 'C2', 'C3'],
#                         'IngridLF2': ['C1', 'C2', 'C3', 'C4', 'C5'], 
#                         'IngridLF3': ['C1', 'C2', 'C3', 'C4'],
#                         'Larry': ['C1', 'C2', 'C3', 'C4', 'C5'],
#                         'MonicaLF1': ['C1', 'C2', 'C3'], 
#                         'MonicaLF2': ['C1', 'C2', 'C3', 'C4', 'C5'],
#                         'George': ['C1', 'C2', 'C3', 'C4', 'C5'], 
#                         'Laurence': ['C1', 'C2', 'C3', 'C4', 'C5'],
#                         'Yasi': ['C2', 'C3', 'C4', 'C5'], 
#                         'Ita' :['C1', 'C2', 'C3', 'C4'],
#                         'Lam': ['C1', 'C2', 'C3', 'C4'], 
#                         'Marcia': ['C1', 'C2', 'C3', 'C4', 'C5'], 
#                         'NathanLF1': ['C1', 'C2', 'C3'], 
#                         'NathanLF2': ['C1', 'C2'], 
#                         'Debbie': ['C2', 'C3', 'C4', 'C5']                          
                        }

# Below is a copy of the full cyclone (wind speed category) list. This list (or part of it) can be copied an pasted
# above and uncommented depending on which cyclone(s) you chose to run.

#                       'IngridLF1': ['C1', 'C2', 'C3'],
#                       'IngridLF2': ['C1', 'C2', 'C3', 'C4', 'C5'], 
#                       'IngridLF3': ['C1', 'C2', 'C3', 'C4'],
#                       'Larry': ['C1', 'C2', 'C3', 'C4', 'C5'],
#                       'MonicaLF1': ['C1', 'C2', 'C3'], 
#                       'MonicaLF2': ['C1', 'C2', 'C3', 'C4', 'C5'],
#                       'George': ['C1', 'C2', 'C3', 'C4', 'C5'], 
#                       'Laurence': ['C1', 'C2', 'C3', 'C4', 'C5'],
#                       'Yasi': ['C2', 'C3', 'C4', 'C5'], 
#                       'Ita' :['C1', 'C2', 'C3', 'C4'],
#                       'Lam': ['C1', 'C2', 'C3', 'C4'], 
#                       'Marcia': ['C1', 'C2', 'C3', 'C4', 'C5'], 
#                       'NathanLF1': ['C1', 'C2', 'C3'], 
#                       'NathanLF2': ['C1', 'C2'], 
#                       'Debbie': ['C2', 'C3', 'C4', 'C5']

## Create NetCDFs for the mangrove cover damage level in Australian Albers tiles
### Setting up functions for creating mangrove damage statistics
This cell sets up some functions for creating NetCDFs of mangrove damage. The code defines some functions that are used when creating the NetCDFs, such as: reading data, characterising the damage to certain canopy classes, and calculating mangrove cover from the Australian Albers tiles and the years of interest. The code then defines and labels the canopy categories based on certain values.

In [ ]:
# Set up some functions for creating NetCDFs of mangrove damage

def read_data_from_netcdf(fname):
    dataset_array = xr.open_dataset(fname)
    data = dataset_array.canopy_cover_class
    #data.values[np.isnan(dataset_array.extent.values)]  = 0
    return data

def categorize_damage(bc_canopy, ac_canopy):
    reduction = bc_canopy - ac_canopy
    return reduction


def main(x1, y1, year):
    mangrove_folder = '/g/data/r78/mangrove_cover/MANGROVE_COVER/'+ '_'.join([str(x1), str(y1)])
    start = int(year) - 1
    end = int(year)
    bc_file = 'MANGROVE_COVER_3577_'  + '_'.join([str(x1), str(y1), str(start)]) + '0101.nc'
    bc_file = path.join(mangrove_folder, bc_file)
    #print("file before cyclone", bc_file)

    bc_canopy = read_data_from_netcdf(bc_file)
    while True:
        ac_file = 'MANGROVE_COVER_3577_'  + '_'.join([str(x1), str(y1), str(end)]) + '0101.nc'
        ac_file = path.join(mangrove_folder, ac_file)
        #print("file after cyclone", ac_file)
        if path.exists(ac_file) == False:
            return

        ac_canopy = read_data_from_netcdf(ac_file)

        result = categorize_damage(bc_canopy.values, ac_canopy.values)
        damage_label = np.zeros(result.shape)
        damage_label[np.isnan(result)] = -1 
        damage_label[result == 1] = 1 # reduce by 1
        damage_label[result == 2] = 2 # reduce by 2
        damage_label[np.logical_and(bc_canopy.values == 2, np.isnan(ac_canopy.values))] = 4 # reduce from 2 
        damage_label[np.logical_and(bc_canopy.values == 3, np.isnan(ac_canopy.values))] = 4 # reduce from 3
        damage_label[np.logical_and(bc_canopy.values == 1, np.isnan(ac_canopy.values))] = 3 # reduce from 1 
        #print(np.where(damage_label == 1))
        #print(np.where(damage_label == 2))
        #print(np.where(damage_label > 2))
        ac_canopy.time.attrs['units'] = "seconds since 1970-01-01 00:00:00"
        result  = xr.Dataset({"damage_level":(['time', 'y', 'x'], damage_label.astype('int16'))}, 
                coords={'time':ac_canopy.time, 'y': ac_canopy.y, 'x':ac_canopy.x},
                attrs={'crs': CRS('EPSG:3577')})
        #print(result)

        dir_name = '.'
        fout = '_'.join(['damage_level', str(x1), str(y1), str(start), str(end)]) + '.nc'
        
        # What about if the file already exists?
        if os.path.isfile(fout) == True:
            print('A file already exists with the same name. We will skip this one')
        else:
            write_dataset_to_netcdf(result, path.join(dir_name, fout))
        end += 1

## Creating mangrove damage statistics
The cell below creates NetCDFs for mangrove damage. This cell will create NetCDFs for each Australian Albers tile for damage from the year before the cyclone to the year after the cyclone, and create NetCDFs that consecutively add one year until 2017. (For example: if the cyclone occured in _2010_, the output files will be for _2009-2010; 2009-2011; 2009-2012... 2009-2017)._ These files measure the per pixel mangrove cover change for the year before (undamaged) and the year(s) after (damaged and recovering) the cyclone. This cell will loop through each of the cyclones defined in the lists above, untill all NetCDFs for those cyclones are created.
### Important note on timing (input year) of cyclone:
Input in the cell below requires the year of which the mangrove damage is greatest following the cyclone. Therefore, the timng of the cyclone must be considered when producing NetCDFs of mangrove cover change. For example: if the cyclone occurred at the start of the year (in the first couple of months), then the year _before_ the cyclone should be used as the _undamaged_ year, and the year _of_ the cyclone as the _damaged_ year (this is true for all but one (Laurence) of the cyclones in the list above, and the actual year of the cyclone can remain unchanged, as these years are the most damaged years). However, if the cyclone occurred at the end of the year (in the last couple of months), then the year _of_ the cyclone should be used as the _undamaged_ year, and the year _after_ the cyclone as _damaged_ year. These considerations must be made before running the cell below, and the year(s) of the cyclones modified in the lists above to produce the correct NetCDFs of mangrove cover change. Therefore, **if a cyclone occurs at the end of the year, then the year in the lists above will have to be modified to one year later so that a NetCDF is produced for the year of the cyclone as the undamaged mangrove cover statistics.** One example of this is cyclone Laurence, which occurred in December 2009, so for the majority of 2009, the area was undamaged. As the most damaged year is required as input when running the below script, in the case of Laurence, 2010 must be used as the input year (despite the cyclone actually occurring in 2009). 

In [ ]:
#Loop through each cyclone
for cyclone in AllCyclones:
    print(cyclone)
    
    ListofTiles = TilesForCyclones[cyclone]
    YearOfCyclone = AllCyclones[cyclone]
    
    # Loop through each tile
    for tile in ListofTiles:
        x, y = tile.split(',')

        # This cell outputs the data for a selected area and time. The first two numbers (int) are the area of interest 
        #from the Australian Albers Polygon layer. The year is the year with the most damage following the cyclone 
        # (see notes above).
        main(int(x),int(y),YearOfCyclone)

## Identify mangrove damage for areas that have been subjected to certain windspeed categories
This cell identifies damage for a mangrove area that has been subjected to a certain wind speed category, during a specific cyclone, by comparing the mangrove cover for the years before and after the cyclone. The output is a csv and a bar chart of the damage level for each of these areas. This cell uses the NetCDF files for mangrove cover from the undamaged to damaged years within each Australian Albers tile for each cyclone. The script manipulates the data in the NetCDF to extract zonal statistics for each specific area. This is a looped cell for running multiple cyclones simultaneously. The cell loops through each cyclone; however, within each cyclone loop, there is a nested loop for the categories, where the script loops through an area that has experienced a certain wind speed category. Within the categories loop, there is a nested loop for the Australian Albers tiles, where the script loops through every NetCDF and extracts damage statistics. The script then converts the damage statistics into km<sup>2</sup>, concatenates all damage data from all tiles for mangroves that experienced specific wind speed categories and then exports plots and csv files of these data. The loop then begins again for the next cyclone/categories/tiles list until all cyclones have been completed.

In [3]:
#Loop through each cyclone
for cyclone in AllCyclones:
    print(cyclone)
    
    ListofTiles = TilesForCyclones[cyclone]
    YearOfCyclone = AllCyclones[cyclone]
    CategoryAll = NumberofCategories[cyclone]
    CategoryLowest = CategoryAll[0]
    CategoryHighest = CategoryAll[-1]
    
    # Loop through each category (nested within the specific cyclone loop)
    for categories in NumberofCategories[cyclone]:
        print(f'working on {cyclone}_{categories}')
        
        # TCName is the name of the folder conatining files for a certain cyclone
        TCName = f'Cyclone_Mangrove_1km_Tiles/{cyclone}_{CategoryLowest}-{CategoryHighest}_{str(YearOfCyclone)}/{cyclone}_{categories}'

        #TCSimulation is the specific name of which shape file is to be run 

        #Note: The 1km mangrove tiles used below have been converted to Australian Albers Projection. This is 
        # performed in a seperate notebook. The script used to convert the tiles to Australian Albers is 
        # the 'Reproject to Albers' script from the 'Mangrove_Stats' notebook.

        TCSimulation = f'{cyclone}_{categories}_albers.shp'
        
        # Make an empty dataframe to concatenate all our results in to 
        ConcatenatedTiles = pd.DataFrame()

        # Loop through each Australian Albers tile (nested within the category loop)
        for tile in ListofTiles:
            x, y = tile.split(',')
            print(f'working on {cyclone}_{x}_{y}')
            
            #TCSimulation2 is the name of the damage NetCDF output from the previous section 
            TCSimulation2 = f'/g/data/r78/sw4446/Mangrove_Change_on_Cyclone/cyclone_damage_results_1yprior/{cyclone}/{cyclone}_{x}_{y}_{YearOfCyclone - 1}_{YearOfCyclone}.nc'
            
            NetCDFDamageData = xr.open_dataset(TCSimulation2)
            
            WindSpeedData = gpd.read_file(f'{TCName}/{TCSimulation}')

            # Calculate damage stats for every 1km tile in the NetCDF and export geojson with counts for each class                  
            out = zonal_stats(f'/g/data/r78/sw4446/Mangrove_Change_on_Cyclone/{TCName}/{TCSimulation}', 
                      f'{TCSimulation2}', 
                      categorical=True, category_map={0: 'No_change', 1: 'Minor_reduction', 2: 'Major_reduction', 
                                                      3: 'Loss_of_woodland', 4: 'Loss_of_forest'}, 
                      geojson_out=True)

            # Convert geojson to Geopandas dataframe and set NaN values to zero
            tile_gpd = gpd.GeoDataFrame.from_features(out).fillna(0)

            # If columns are missing data, this script creates columns (e.g. 'No_change', 'Minor_reduction', 
            #'Major_reduction', 'Loss_of_woodland', 'Loss_of_forest') so that the following parts of the script can run.
            try:
                tile_gpd['total'] = tile_gpd[['No_change', 'Minor_reduction', 'Major_reduction', 'Loss_of_woodland', 'Loss_of_forest']].sum(axis=1)
            except KeyError:
                if 'No_change' not in tile_gpd:
                    #print('Adding No_change')
                    tile_gpd['No_change'] = 0
                    
            try:
                tile_gpd['total'] = tile_gpd[['No_change', 'Minor_reduction', 'Major_reduction', 'Loss_of_woodland', 'Loss_of_forest']].sum(axis=1)
            except KeyError:
                if 'Minor_reduction' not in tile_gpd:
                    #print('Adding Minor_reduction')
                    tile_gpd['Minor_reduction'] = 0
               
            try:
                tile_gpd['total'] = tile_gpd[['No_change', 'Minor_reduction', 'Major_reduction', 'Loss_of_woodland', 'Loss_of_forest']].sum(axis=1)
            except KeyError:
                if 'Major_reduction' not in tile_gpd:
                    #print('Adding Major_reduction')
                    tile_gpd['Major_reduction'] = 0
               
            try:
                tile_gpd['total'] = tile_gpd[['No_change', 'Minor_reduction', 'Major_reduction', 'Loss_of_woodland', 'Loss_of_forest']].sum(axis=1)
            except KeyError:
                if 'Loss_of_woodland' not in tile_gpd:
                    #print('Adding Loss_of_woodland')
                    tile_gpd['Loss_of_woodland'] = 0
               
            try:
                tile_gpd['total'] = tile_gpd[['No_change', 'Minor_reduction', 'Major_reduction', 'Loss_of_woodland', 'Loss_of_forest']].sum(axis=1)
            except KeyError:
                if 'Loss_of_forest' not in tile_gpd:
                    #print('Adding Loss_of_forest')
                    tile_gpd['Loss_of_forest'] = 0
                
  
            # This code sums up all observations for each wind speed category within each tile for each cyclone.
            # Sum up the total number of observed changes for each Australian Albers tile
            tile_gpd['SumofObs'] = tile_gpd[['No_change', 'Minor_reduction', 'Major_reduction', 'Loss_of_woodland', 'Loss_of_forest']].sum(axis=1)

            # Re-rearrange data to match required file format
            GetRidofExtraData = tile_gpd[['ID','No_change', 'Minor_reduction', 'Major_reduction', 
                                 'Loss_of_woodland', 'Loss_of_forest']]

            TidiedCategoriesinkm2 = GetRidofExtraData
            TidiedCategoriesinkm2['No_change_km2'] = (GetRidofExtraData['No_change']) * 625 / 1000000
            TidiedCategoriesinkm2['Minor_reduction_km2'] = (GetRidofExtraData['Minor_reduction']) * 625 / 1000000
            TidiedCategoriesinkm2['Major_reduction_km2'] = (GetRidofExtraData['Major_reduction']) * 625 / 1000000
            TidiedCategoriesinkm2['Loss_of_woodland_km2'] = (GetRidofExtraData['Loss_of_woodland']) * 625 / 1000000
            TidiedCategoriesinkm2['Loss_of_forest_km2'] = (GetRidofExtraData['Loss_of_forest']) * 625 / 1000000

            # Formatting name to remove unnecessary name parts
            TCNameList = re.sub('_albers.shp', '', TCSimulation)        

            # Exporting data as csv to wind speed category folder for specific cyclone 
            TidiedCategoriesinkm2.to_csv(f'{TCName}/{TCNameList}_{x}_{y}_ImmediateMangroveImpact.csv')

            out_list = []

            # Combine into a single pandas dataframe, then write to pickle
            TidiedCategoriesinkm2.to_pickle(f'/g/data/r78/sw4446/Mangrove_Change_on_Cyclone/{TCName}/MangChange_{TCNameList}_1km_1987_to_2017.pkl.gz', 
                                    compression="gzip")

            dataFile = f'/g/data/r78/sw4446/Mangrove_Change_on_Cyclone/{TCName}/MangChange_{TCNameList}_1km_1987_to_2017.pkl.gz'
            data = pd.read_pickle(dataFile, compression="gzip")
            data['total'] = data[['No_change_km2', 'Minor_reduction_km2', 'Major_reduction_km2', 
                                 'Loss_of_woodland_km2', 'Loss_of_forest_km2']].sum(axis=1)
            
            # Concatenate data from each tile 
            ConcatenatedTiles = pd.concat([ConcatenatedTiles, TidiedCategoriesinkm2], axis=0, join='outer', ignore_index=True)

            
        # Plotting mangrove cover damage data for each wind speed category experienced during specified cyclones     
        print(f'plotting {cyclone}_{categories}')
        
        # Output location for plotted data
        OutPlotLocation = f'{TCName}/{TCNameList}_Linear.jpg'
        
        outData = ConcatenatedTiles.sum(axis=0)[['No_change_km2', 'Minor_reduction_km2', 'Major_reduction_km2', 
                             'Loss_of_woodland_km2', 'Loss_of_forest_km2']]
        
        # The code below sets up some styles and parameters for plotting. Here, the colours of the bars, 
        # the x-axis labels, font sizes, plot area and other features of the plots can be modified. 
        
        ax=outData.plot.bar(stacked=False,
                                                figsize=(10,10), color=['#27AE60', '#F9A56C', '#F2734A', '#D93C27', '#B21F24'],
                                                legend=False, width=1.0)
        xlabels = ['No change', 'Minor reduction', 'Major reduction', 
                         'Loss of woodland', 'Loss of forest']
        ax.set_xticklabels(xlabels, rotation = 45, fontsize=12)
        (subplots_adjust(bottom=0.30))
        ax.set_xlabel('Damage categories', fontsize=14)
        ax.set_ylabel('Area (km$^{2}$)', fontsize=14)
        ax.set_title(f'{cyclone}_{categories}', fontsize=16, fontweight='bold')
        
        # For plots with logarithmic y-axes, uncomment the code below. 
        # (The current output is for plots with linear y-axes).  
        
        #ax.set_yscale('log')
        
        # Saving, clearing and closing plots
        plt.savefig(OutPlotLocation)
        plt.clf()
        plt.close()

print('All done!')

IngridLF1
working on IngridLF1_C1
working on IngridLF1_12_-14


/g/data/v10/public/modules/dea-env/20181015/lib/python3.6/site-packages/ipykernel/__main__.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/g/data/v10/public/modules/dea-env/20181015/lib/python3.6/site-packages/ipykernel/__main__.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/g/data/v10/public/modules/dea-env/20181015/lib/python3.6/site-packages/ipykernel/__main__.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

working on IngridLF1_12_-15


/g/data/v10/public/modules/dea-env/20181015/lib/python3.6/site-packages/ipykernel/__main__.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/g/data/v10/public/modules/dea-env/20181015/lib/python3.6/site-packages/ipykernel/__main__.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/g/data/v10/public/modules/dea-env/20181015/lib/python3.6/site-packages/ipykernel/__main__.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

working on IngridLF1_12_-16


/g/data/v10/public/modules/dea-env/20181015/lib/python3.6/site-packages/ipykernel/__main__.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/g/data/v10/public/modules/dea-env/20181015/lib/python3.6/site-packages/ipykernel/__main__.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/g/data/v10/public/modules/dea-env/20181015/lib/python3.6/site-packages/ipykernel/__main__.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

working on IngridLF1_13_-16


KeyboardInterrupt: 

In [5]:
tile_gpd

,ID,X_MAX,X_MIN,Y_MAX,Y_MIN,geometry,max_1,-1,No_change,Minor_reduction,Major_reduction,Loss_of_forest,Loss_of_woodland,total,SumofObs
0,1835050,143.93242,143.92242,-14.561896,-14.571896,POLYGON ((1296094.729371981 -1604080.846721412...,20.098410,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1835051,143.94242,143.93242,-14.561896,-14.571896,POLYGON ((1297178.674784514 -1604182.340118145...,20.126259,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1835052,143.95242,143.94242,-14.561896,-14.571896,POLYGON ((1298262.612249397 -1604283.918359695...,20.518696,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1835053,143.96242,143.95242,-14.561896,-14.571896,POLYGON ((1299346.541759985 -1604385.581445432...,21.650633,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1835054,143.97242,143.96242,-14.561896,-14.571896,POLYGON ((1300430.463309644 -1604487.329374738...,22.378183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1835055,143.98242,143.97242,-14.561896,-14.571896,POLYGON ((1301514.376891728 -1604589.162146989...,20.226343,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1835056,143.99242,143.98242,-14.561896,-14.571896,"POLYGON ((1302598.282499596 -1604691.07976156,...",20.248459,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1835057,144.00242,143.99242,-14.561896,-14.571896,"POLYGON ((1303682.18012661 -1604793.082217826,...",20.270100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1835058,144.01242,144.00242,-14.561896,-14.571896,POLYGON ((1304766.069766128 -1604895.169515164...,20.290075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1835059,144.02242,144.01242,-14.561896,-14.571896,POLYGON ((1305849.951411506 -1604997.341652947...,20.309565,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Combined damage statistics for wind speed categories
#### Combined damage statistics for mangrove areas that have been subjected to certain wind speed categories for all cyclones
This cell uses the damage statistics csv files produced in the previous (`Identify mangrove damage for areas that have been subjected to certain windspeed categories`) cell to create plots and csv files for the combined damage statistics for each wind speed category for all cyclones. This is achieved by calculating the sum of damage from each wind speed category for all cyclones. The outputs of this cell are bar charts and csv files of the total damage for each category combined for all cyclones. This cell will loop through and export statistics for each area affected by certain wind speed category defined in the `categories` list until completed.

In [ ]:
# Plot combined damage for each wind speed category for all cyclones

# Setting up base folder location
baseFolder = '/g/data/r78/sw4446/Mangrove_Change_on_Cyclone/Cyclone_Mangrove_1km_Tiles'

# Category list for use in the loop below
categories = ['C1', 'C2', 'C3', 'C4', 'C5']

# Loop runs through the csv files of damage and concatenates the data for all cyclones by category
for category in categories:
    print(f'creating new dataframe for category: {category}')
    sum_df = pd.DataFrame()
    for dirpath, dirnames, filenames in os.walk(baseFolder):
        for filename in [f for f in filenames if f.endswith("ImmediateMangroveImpact.csv")]:
            if category in filename:
                print(f'adding {filename} to dataframe')
                df = pd.read_csv (os.path.join(dirpath, filename))
                sum_df = pd.concat([sum_df, df], axis=0, join='outer', ignore_index=True)
    print('writing dataframe to csv')
    # csv export directory
    sum_df.to_csv(baseFolder + f'/{category}_sum_of_cyclone.csv')
    
    outData = sum_df.sum(axis=0)[['No_change_km2', 'Minor_reduction_km2', 'Major_reduction_km2', 
                     'Loss_of_woodland_km2', 'Loss_of_forest_km2']]
    
    # Setting up some functions for plotting. Values in the script below can be adjusted to modify plot styles.
    ax=outData.plot.bar(stacked=False,
                                            figsize=(10,10), color=['#27AE60', '#F9A56C', '#F2734A', '#D93C27', '#B21F24'],
                                            legend=False, width=1.0)

    xlabels = ['No change', 'Minor reduction', 'Major reduction', 
                     'Loss of woodland', 'Loss of forest']

    ax.set_xticklabels(xlabels, rotation = 45, fontsize=12)
    ax.set_xlabel('Damage categories', fontsize=14)
    ax.set_ylabel('Area (km$^{2}$)', fontsize=14)
    ax.set_title(f'{category}', fontsize=16, fontweight='bold')
    
    # For plots with logarithmic y-axes, uncomment the code below. 
    # (The current output is for plots with linear y-axes).  
        
    #ax.set_yscale('log')
    
    plt = ax.get_figure()

    # Plot export directory
    plt.savefig(baseFolder + f'/{category}_sum_of_cyclone_LinearY.jpeg')

    plt.clf()
    
print('All Done!')

## Total damage statistics for each cyclone
This cell uses the damage statistics csv files produced in the previous (`Identify mangrove damage for areas that have been subjected to certain windspeed categories`) cell to create plots and csv files of total damage for each cyclone. This is achieved by calculating the sum of damage each cyclone. The outputs of this cell are bar charts and csv files of the total damage for each cyclone. This cell will search through the folders for each cyclone, extract the relevant damage statistics data from each csv and calcutlate the total damage for each cyclone. This cell will loop through and export statistics for each cyclone defined in the `cyclones` list until completed.

In [ ]:
# Create plot and csv for total damage for each cyclones

# Setting up base folder location

baseFolder = '/g/data/r78/sw4446/Mangrove_Change_on_Cyclone/Cyclone_Mangrove_1km_Tiles'

# Cyclone list for use in the loop below.
cyclones = ['IngridLF1', 'IngridLF2', 'IngridLF3', 'Larry', 'MonicaLF1', 'MonicaLF2','George', 'Laurence', 'Yasi', 
                'Ita', 'Lam', 'Marcia', 'NathanLF1', 'NathanLF2', 'Debbie']
                
# Loop runs through the csv files of damage and concatenates the data for each cyclone
for cyclone in cyclones:
    print(f'creating new dataframe for cyclone: {cyclone}')
    sum_df = pd.DataFrame()
    for dirpath, dirnames, filenames in os.walk(baseFolder):
        for filename in [f for f in filenames if f.endswith("ImmediateMangroveImpact.csv")]:
            if cyclone in filename:
                print(f'adding {filename} to dataframe')
                df = pd.read_csv (os.path.join(dirpath, filename))
                sum_df = pd.concat([sum_df, df], axis=0, join='outer', ignore_index=True)
    print('writing dataframe to csv')
    # csv export directory
    sum_df.to_csv(baseFolder + f'/{cyclone}_sum_of_cyclone_DamageOnly.csv')
    
    outData = sum_df.sum(axis=0)[['Minor_reduction_km2', 'Major_reduction_km2', 
                     'Loss_of_woodland_km2', 'Loss_of_forest_km2']]

    # Setting up some functions for plotting. Values in the script below can be adjusted to modify plot styles.    
    ax=outData.plot.bar(stacked=False,
                                            figsize=(10,10), color=['#F9A56C', '#F2734A', '#D93C27', '#B21F24'],
                                            legend=False, width=1.0)

    xlabels = ['Minor reduction', 'Major reduction', 
                     'Loss of woodland', 'Loss of forest']

    ax.set_xticklabels(xlabels, rotation = 45, fontsize=12)
    (subplots_adjust(bottom=0.20))
    ax.set_xlabel('Damage categories', fontsize=14)
    ax.set_ylabel('Area (km$^{2}$)', fontsize=14)
    ax.set_title(f'{cyclone}', fontsize=16, fontweight='bold')

    # For plots with logarithmic y-axes, uncomment the code below. 
    # (The current output is for plots with linear y-axes).  
        
    #ax.set_yscale('log')    

    plt = ax.get_figure()
    
    # Plot export directory
    plt.savefig(baseFolder + f'/{cyclone}_sum_of_cyclone_DamageOnly_LinearY.jpeg')

    plt.clf()
    
print('All Done!')